# 네이버 영화 크롤링 3

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

In [2]:
test_url = "https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136990&type=after&page=1"
resp = requests.get(test_url)
html = BeautifulSoup(resp.content, 'html.parser')
html


<!DOCTYPE html>

<html lang="ko">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<title>네이버 영화</title>
<link href="https://ssl.pstatic.net/static/m/movie/icons/naver_movie_favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/css/common.css?20200515103320" rel="stylesheet" type="text/css">
<link href="/css/movie_tablet.css?20200515103320" rel="stylesheet" type="text/css"/>
<link href="/css/movie_end.css?20200515103320" rel="stylesheet" type="text/css"/>
<script src="/js/deploy/movie.all.js?20200515103320" type="text/javascript"></script>
</link></head>
<body>
<!-- content -->
<input id="movieCode" name="movieCode" type="hidden" value="136990"/>
<input id="onlyActualPointYn" name="onlyActualPointYn" type="hidden" value="N"/>
<input id="includeSpoilerYn" name="includeSpoilerYn" type="hidden" value="N"/>
<input id="order" name="order" type="hidden" value="sympathyScore"/>
<input id="page" name="page" type="hidden" value="1"/>
<div clas

In [3]:
score_result = html.find('div', {'class': 'score_result'})
lis = score_result.findAll('li')
lis[0]

<li>
<div class="star_score">
<span class="st_off"><span class="st_on" style="width:100.0%"></span></span><em>10</em>
</div>
<div class="score_reple">
<p>
<!-- 스포일러 컨텐츠로 처리되는지 여부 -->
<span id="_filtered_ment_0">
														
															
															
																처음에 만두 같은게 나와서 상영관 잘못 들어온줄 
															
														
														
													</span>
</p>
<dl>
<dt>
<em>
<a href="#" onclick="javascript:showPointListByNid(14315731, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
<span>크리스탈(a8d5****)</span>
</a>
</em>
<em>2018.07.18 09:07</em>
</dt>
<dd>
<a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','a8d5****', 'aP0Bff3X91mms0YG5+wXosU0Ei+cOdrTi7lVXvYU+Lo=', '처음에 만두 같은게 나와서 상영관 잘못 들어온줄 ', '14315731', 'point_after', false);return false;"><em>신고</em></a>
</dd>
</dl>
</div>
<div class="btn_area">
<a class="_sympathyButton" href="#" onclick="parent.clickcr(this, 'ara.sym'

In [4]:
review_text = lis[0].find('p').getText()
review_text

'\n\n\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t처음에 만두 같은게 나와서 상영관 잘못 들어온줄 \r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t\t\t\n'

In [5]:
score = lis[0].find('em').getText()
score

'10'

In [6]:
like = lis[0].find('div', {'class': 'btn_area'}).findAll('span')[1].getText()
dislike = lis[0].find('div', {'class': 'btn_area'}).findAll('span')[3].getText()
like, dislike

('공감', '비공감')

In [7]:
nickname = lis[0].findAll('a')[0].find('span').getText()
nickname

'크리스탈(a8d5****)'

In [97]:
def get_data(url):
    resp = requests.get(url)
    html = BeautifulSoup(resp.content, 'html.parser')
    score_result = html.find('div', {'class': 'score_result'})
    lis = score_result.findAll('li')
    lis_df = pd.DataFrame()
    for li in lis:
        nickname = li.findAll('a')[0].find('span').getText() if li.findAll('a')[0].find('span') is not None else None
        created_at = datetime.strptime(li.find('dt').findAll('em')[-1].getText(), "%Y.%m.%d %H:%M")
        review_text = li.find('p').getText().translate(str.maketrans({"\n": "", "\r": "", "\t": ""}))
        score = li.find('em').getText()
        btn_likes = li.find('div', {'class': 'btn_area'}).findAll('strong')
        like = btn_likes[0].getText()
        dislike = btn_likes[1].getText()
        watch_movie = li.find('span', {'class':'ico_viewer'})
        
        li_df = pd.DataFrame({"nickname": [nickname],
                            "review": [review_text],
                            "score": [score],
                            "like": [like],
                            "dislike": [dislike],
                            "created at": [created_at],
                            "watch_movie": [watch_movie and True or False]})
        lis_df = pd.concat([lis_df, li_df])
    return(lis_df)

In [100]:
test_url = 'https://movie.naver.com/movie/bi/mi/pointWriteFormList.nhn?code=136990&type=after'
resp = requests.get(test_url)
html = BeautifulSoup(resp.content, 'html.parser')
result = html.find('div', {'class':'score_total'}).find('strong').findChildren('em')[-1].getText()
total_count = int(result.replace(',', ''))

rslt = pd.DataFrame()
for i in range(1, int(total_count / 10) + 1):
    url = test_url + '&page=' + str(i)
#     print('process parsing: url="' + url + '"')
    get_data(url)
    tmp = get_data(url)
    rslt = pd.concat([rslt, tmp])

In [101]:
rslt.head(100)

,nickname,review,score,like,dislike,created at,watch_movie
0,크리스탈(a8d5****),처음에 만두 같은게 나와서 상영관 잘못 들어온줄,10,2555,155,2018-07-18 09:07:00,False
0,gksq****,14년동안 기다린 보람이있다.인크레더블 3도 해주세요 ㅠㅠ 제발 ㅠㅠ,10,1735,72,2018-07-18 10:39:00,False
0,yski****,잭잭이랑 에드나 케미 미쳤닼ㅋㅋㅋ,10,1524,41,2018-07-18 10:07:00,False
0,배센도(bbtj****),어릴 때 1편을 보고 성인이 된 올해 2편을 봤다. 또 보고 싶다. 3편도 나오면 ...,10,1161,38,2018-07-18 12:28:00,False
0,space(tmd5****),속편도 이리 완벽할 수 있구나..!,10,1046,46,2018-07-18 09:27:00,False
...,...,...,...,...,...,...,...
0,도승지(haku****),관람객만두에서울뻔함....ㅠㅠ,10,6,0,2018-07-27 19:03:00,True
0,woos****,더빙판 추천 이래서 전문 성우를 써야됨,10,6,0,2018-07-26 10:25:00,False
0,None,개인적으로 모든 애니메이션 영화중에서 가장 재미있어요. 속편이 이렇게 완벽하다니 인...,10,6,0,2018-07-26 01:00:00,False
0,링규(byor****),스피드감좋고 전작보다 재미있고 스토리도 바로이어지고 무지잼낫음,10,6,0,2018-07-24 07:01:00,False


In [103]:
rslt.shape

(10260, 7)